# Stitch
---

#### Overview
Interactive stitching of one or several sections.

In [2]:
# indirectly enable autocomplete
%config Completer.use_jedi = False

# autoreload modules
%load_ext autoreload
%autoreload 2

In [3]:
import pathlib
import requests

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import renderapi

from scripted_render_pipeline import basic_auth
from interactive_render import plotting

#### `render-ws` environment variables

In [4]:
# # render-ws environment variables
# params_render = {
#     "host": "https://sonic.tnw.tudelft.nl",
#     "owner": "fastem",
#     "project": "20230712_RF_zstack",
#     "auth": basic_auth.load_auth()
# }

# create an authorized session
auth = basic_auth.load_auth()
sesh = requests.Session()
sesh.auth = auth

# render-ws environment variables
params_render = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "session": sesh
}

params_align = {
    "host": "http://localhost",
    "port": 8081,
    "client_scripts": "/home/catmaid/render/render-ws-java-client/src/main/scripts",
    "client_script": "/home/catmaid/render/render-ws-java-client/src/main/scripts/run_ws_client.sh",
    "owner": "akievits",
    "project": "20230914_RP_exocrine_partial_test",
    "memGB": '40G', # Allocate enough memory
    "session": sesh
}

# set project directory
dir_project = pathlib.Path("/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/")

## Stitching (part I)
---
Get tile pairs. Get point matches.

### Get tile pairs

Use render-ws `tilePairClient` to get the set of tile pairs.

In [5]:
# choose stack from which to get tile pairs
stack = "postcorrection"
z_values = [int(z) for z in renderapi.stack.get_z_values_for_stack(
    stack,
    **params_render
)]

# Get tile pairs from the rough aligned stack
tilepairs = renderapi.client.tilePairClient(
    stack=stack,
    minz=min(z_values),
    maxz=max(z_values),
    zNeighborDistance=0,  # half-height of search cylinder
    excludeSameLayerNeighbors=False,
    subprocess_mode="check_output",  # suppresses output
    **params_render
)["neighborPairs"]

# Reformat tilepairs for PointMatchClient
tilepairs = [[tp['p']['id'], tp['q']['id']] for tp in tilepairs]

# Show tile pairs
out = f"Number of tile pairs... {len(tilepairs)}"
print(out, "\n" + "-"*len(out))
tilepairs[:5]

Number of tile pairs... 120 
---------------------------


[['t00_z0_y4_x4', 't05_z0_y3_x4'],
 ['t00_z0_y4_x4', 't01_z0_y4_x3'],
 ['t01_z0_y4_x3', 't06_z0_y3_x3'],
 ['t01_z0_y4_x3', 't02_z0_y4_x2'],
 ['t02_z0_y4_x2', 't07_z0_y3_x2']]

### Get point matches

Use `render-ws` `PointMatchClient` script to find matching features between the neighboring megafields.

In [5]:
from renderapi.client import (
    SiftPointMatchOptions,
    MatchDerivationParameters,
    FeatureExtractionParameters
)

In [6]:
# Name for pointmatch collection
match_collection = f"{params_render['project']}_{stack}_matches"
match_collection

'20230914_RP_exocrine_partial_test_postcorrection_matches'

#### Set SIFT + RANSAC parameters

In [7]:
# `RANSAC` parameters
params_RANSAC = MatchDerivationParameters(
    matchIterations=None,
    matchMaxEpsilon=25,        # maximal alignment error
    matchMaxNumInliers=None,
    matchMaxTrust=None,
    matchMinInlierRatio=0.05,  # minimal inlier ratio
    matchMinNumInliers=7,      # minimal number of inliers
    matchModelType='AFFINE',   # expected transformation
    matchRod=0.92              # closest/next closest ratio
)

# `SIFT` parameters
params_SIFT = FeatureExtractionParameters(
    SIFTfdSize=8,              # feature descriptor size
    SIFTmaxScale=0.30,         # (width/height *) maximum image size
    SIFTminScale=0.20,         # (width/height *) minimum image size
    SIFTsteps=5                # steps per scale octave
)

# Combined `SIFT` & `RANSAC` parameters
params_SIFT = SiftPointMatchOptions(
    fillWithNoise=True,
    **{**params_RANSAC.__dict__,
       **params_SIFT.__dict__}
)

# Extra parameters
params_SIFT.numberOfThreads = 20  # multithreading
params_SIFT.clipWidth = 640      # N pixels included in rendered clips of LEFT/RIGHT oriented montage tiles
params_SIFT.clipHeight = 640     # N pixels included in rendered clips of TOP/BOTTOM oriented montage tiles
params_SIFT.__dict__

{'SIFTfdSize': 8,
 'SIFTmaxScale': 0.3,
 'SIFTminScale': 0.2,
 'SIFTsteps': 5,
 'matchIterations': None,
 'matchMaxEpsilon': 25,
 'matchMaxNumInliers': None,
 'matchMaxTrust': None,
 'matchMinInlierRatio': 0.05,
 'matchMinNumInliers': 7,
 'matchModelType': 'AFFINE',
 'matchRod': 0.92,
 'renderScale': None,
 'fillWithNoise': True,
 'numberOfThreads': 20,
 'clipWidth': 640,
 'clipHeight': 640}

#### Execute SIFT + RANSAC

In [9]:
# Run SIFT + RANSAC via render-ws PointMatchClient
renderapi.client.pointMatchClient(
    stack=stack,
    collection=match_collection,
    tile_pairs=tilepairs,
    sift_options=params_SIFT,
    excludeAllTransforms=True,
    subprocess_mode='check_output',  # suppresses output
    **params_align
    )

  0%|          | 0/120 [00:00<?, ?it/s]

### Inspect

In [13]:
from interactive_render import utils

In [16]:
plotting.plot_stack_with_stitching_matches(
    stack,
    match_collection,
    params_render,
    width=256
)

interactive(children=(IntSlider(value=0, description='z', max=2), Output()), _dom_classes=('widget-interact',)…

## Stitching (part II)
---

### Create alignment files

In [17]:
from pathlib import Path
import os
import subprocess
import json
from pprint import pprint

In [18]:
# Load align.json template
template_align_json = Path('../templates/montage.json')
with template_align_json.open('r') as json_data:
    params_align = json.load(json_data)

# Loop through sections
for z in tqdm(z_values):

    # Edit BigFeta solver schema
    params_align['first_section'] = z
    params_align['last_section'] = z
    params_align['transformation'] = 'TranslationModel'

    # Edit input stack data
    params_align['input_stack']['host'] = params_render['host']
    params_align['input_stack']['owner'] = params_render['owner']
    params_align['input_stack']['project'] = params_render['project']
    params_align['input_stack']['name'] = stack

    # Edit point match stack data
    params_align['pointmatch']['host'] = params_render['host']
    params_align['pointmatch']['owner'] = params_render['owner']
    params_align['pointmatch']['name'] = match_collection

    # Edit output stack data
    params_align['output_stack']['host'] = params_render['host']
    params_align['output_stack']['owner'] = params_render['owner']
    params_align['output_stack']['project'] = params_render['project']
    params_align['output_stack']['name'] = f"{stack}_stitched"

    # Edit regularization parameters
    params_align['regularization']['default_lambda'] = 0.005      # default: 0.005
    params_align['regularization']['translation_factor'] = 0.005  # default: 0.005

    # Export montage settings to
    sectionId = renderapi.stack.get_sectionId_for_z(stack, z, **params_render)
    align_json = dir_project / f"{sectionId}/{stack}/montage.json" if stack == "corrected" else \
                 dir_project / f"{sectionId}/montage.json"
    with align_json.open('w') as json_data:
        json.dump(params_align, json_data, indent=2)

# Check alignment parameters
print(align_json)
print('-'*len(align_json.as_posix()))
pprint(params_align)

  0%|          | 0/3 [00:00<?, ?it/s]

/long_term_storage/akievits/FAST-EM/20230914_RP_exocrine_partial_test/S005/montage.json
---------------------------------------------------------------------------------------
{'close_stack': 'True',
 'first_section': 2,
 'hdf5_options': {'chunks_per_file': -1, 'output_dir': ''},
 'input_stack': {'client_scripts': '/home/catmaid/render/render-ws-java-client/src/main/scripts',
                 'collection_type': 'stack',
                 'db_interface': 'render',
                 'host': 'http://localhost',
                 'name': 'postcorrection',
                 'owner': 'akievits',
                 'port': 8081,
                 'project': '20230914_RP_exocrine_partial_test'},
 'last_section': 2,
 'log_level': 'INFO',
 'matrix_assembly': {'cross_pt_weight': 1.0,
                     'depth': 2,
                     'inverse_dz': 'True',
                     'montage_pt_weight': 1.0,
                     'npts_max': 500,
                     'npts_min': 5},
 'output_mode': 'stack',


### Compute optimal transformations with `BigFeta`

In [19]:
# Path to `BigFeta`
cwd = Path.cwd().as_posix()
dir_BigFeta = Path('/home/akievits/BigFeta/')

# Loop through z values
for z in z_values:

    # Select montage json
    sectionId = renderapi.stack.get_sectionId_for_z(stack, z, **params_render)
    align_json = dir_project / f"{sectionId}/{stack}/montage.json" if stack == "corrected" else \
                 dir_project / f"{sectionId}/montage.json"

    # Call `BigFeta.BigFeta` process -- have to switch to BigFeta directory
    os.chdir(dir_BigFeta.as_posix())
    subprocess.run(['python', '-m', 'bigfeta.bigfeta', '--input_json', align_json.as_posix()])
    os.chdir(cwd)

/home/akievits/interactive-render-workflow/.venv/lib/python3.9/site-packages/argschema/utils.py:346: FutureWarning: '--transform_apply' is using old-style command-line syntax with each element as a separate argument. This will not be supported in argschema after 2.0. See http://argschema.readthedocs.io/en/master/user/intro.html#command-line-specification for details.
  warnings.warn(warn_msg, FutureWarning)
INFO:bigfeta.utils:
 loaded 25 tile specs from 1 zvalues in 0.0 sec using interface: render
INFO:__main__: A created in 0.2 seconds
INFO:__main__:
 solved in 0.0 sec
 precision [norm(Kx-Lm)/norm(Lm)] = 8.5e-16, 6.5e-16
 error     [norm(Ax-b)] = 553.001, 599.439
 [mean(Ax) +/- std(Ax)] : 0.0 +/- 5.8, 0.0 +/- 6.3
 [mean(error mag) +/- std(error mag)] : 7.5 +/- 4.2
INFO:__main__:
 scales: 1.00 +/- 0.00, 1.00 +/- 0.00
INFO:bigfeta.utils:
ingesting results to http://localhost:8081 akievits__20230914_RP_exocrine_partial_test__postcorrection_stitched
INFO:bigfeta.utils:render output is goi

### Inspect

In [22]:
# Plot stacks
stacks = ["postcorrection", "postcorrection_stitched"]
plotting.plot_stacks(
    stacks,
    **params_render
)

  0%|          | 0/2 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=2), IntSlider(value=25165, description='vmin', m…